# Set constants

In [1]:
test_data_path = "./tmp/test.csv"

# Load test data

In [2]:
import yfinance as yf
import pandas as pd
from NNTrade.common.candle_col_name import INDEX, OPEN,CLOSE, HIGH, LOW, VOLUME

In [3]:
import os

if not os.path.exists(test_data_path):
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2020-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = INDEX
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.rename({"Open":OPEN, "High": HIGH, "Low":LOW, "Close":CLOSE, "Volume":VOLUME},axis=1, inplace=True)
    data_df.to_csv(test_data_path)

In [4]:
data_df:pd.DataFrame = pd.read_csv(test_data_path, index_col=INDEX)
data_df.index = pd.to_datetime(data_df.index)
data_df

,open,high,low,close,volume,Dividends,Stock Splits
start_date_time,,,,,,,
2020-01-01,1.122083,1.122838,1.115947,1.122083,10000000,0.0,0.0
2020-01-02,1.121894,1.122712,1.116682,1.122083,10000000,0.0,0.0
2020-01-03,1.117081,1.118068,1.112570,1.117144,10000000,0.0,0.0
2020-01-06,1.116246,1.120825,1.115810,1.116196,10000000,0.0,0.0
2020-01-07,1.119583,1.119946,1.113487,1.119799,10000000,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0
2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0
2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0


# Set up data source

In [5]:
data_source = {"main":data_df}

# Set strategy

In [6]:
from pyalgotrade.technical import ma
from src.common.candle_data_set_config import CandleDataSetConfig
from src.simulation.config.strategy_config import StrategyConfig
from src.simulation.realization.pyalgotrade.strategy_wrapper import PyalgotradeStrategyWrapper
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from pyalgotrade.broker import OrderExecutionInfo, Order
from src.simulation.models import Deal

class MyStrategy(PyalgotradeStrategyWrapper):
    def __init__(self, barFeed,  strategy_config: StrategyConfig):
        super().__init__(barFeed, strategy_config)
        # We want a 15 period SMA over the closing prices.
        self.__instrument = "main"
        self.__sma = ma.SMA(barFeed[self.__instrument].getCloseDataSeries(), 15)
        self.__position:Position = None
        self.__last_deal:Deal = None
        self.__signal = float(strategy_config["delta"])

    def _action_to_stirng(self, action):
        if action == Order.Action.BUY:
            return "BUY"
        elif action == Order.Action.SELL:
            return "SELL"
        elif action == Order.Action.SELL_SHORT:
            return "SELL_SHORT"
        elif action == Order.Action.BUY_TO_COVER:
            return "BUY_TO_COVER"
        else:
            raise Exception(f"Unexpected action {action}")

    def onEnterOk(self, position:Position):
        order: Order = position.getEntryOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()

        self.info(f"{self._action_to_stirng(order.getAction())} at {execInfo.getPrice()}")
        self.__last_deal = Deal.BuildFromCap(execInfo.getDateTime(), execInfo.getPrice(), execInfo.getQuantity(), order.getInstrument(), self.__get_cur_equity())
        self._deal_list.append(self.__last_deal)

    def onEnterCanceled(self, position:Position):
        self.__position = None

    def onExitOk(self, position:Position):
        order: Order = position.getExitOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()
        self.info(f"Close {self._action_to_stirng(order.getAction())} order at {execInfo.getPrice()}")
        self.__position = None
        self.__last_deal.close_deal(execInfo.getDateTime(), execInfo.getPrice())
        self.__last_deal = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def __get_cur_equity(self)->float:
        return self.getBroker().getEquity()

    def onBars(self, bars):
        equity = self.__get_cur_equity()

        bar: BasicBar = bars[self.__instrument]
        
        if self.__last_deal is not None:
            self.__last_deal.set_last_price(bar.getPrice())

        self._cap_log[bar.getDateTime()] = equity

        if self.__sma[-1] is None:
            return
        
        
        if bar.getPrice()/self.__sma[-1] - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 100000, True)
            elif isinstance(self.__position, ShortPosition) and not self.__position.exitActive():
                self.__position.exitMarket()
        elif self.__sma[-1]/bar.getPrice() - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterShort(self.__instrument, 100000, True)
            elif isinstance(self.__position, LongPosition) and not self.__position.exitActive():
                self.__position.exitMarket()


# Optimizing

In [7]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig
from NNTrade.common import TimeFrame

In [8]:
from datetime import date
from src.common.candle_config import CandleConfig
from src.common.candle_data_set_config import CandleDataSetConfig
from src.common.date_period import DatePeriod
from src.optimization.config.strategy_config_sets import StrategyConfigSet


opt_config = OptimizationConfig(
    CandleDataSetConfig.BuildFrom(CandleConfig("EURUSD",TimeFrame.D),TimeFrame.D,"main"), 
    DatePeriod(date(2020,1,1), date(2024,1,1)),
    StrategyConfigSet({"delta":[0,0.0001,0.0005, 0.0010, 0.005, 0.01,]}))

In [9]:
from src.simulation.realization.pyalgotrade.pyalgotrade_trading_simulator import PyalgotradeTradingSimulator,Frequency
from src.optimization_analyzer.period_splitter import DefaultPeriodSplitter,timedelta

trading_simulator = PyalgotradeTradingSimulator(data_source, lambda bf, str_cfg:MyStrategy(bf,str_cfg))

In [10]:
opt_analyzer = OptimizationAnalyzer(trading_simulator, period_splitter=DefaultPeriodSplitter.from_days(360, 90))
analyzation_report_arr = opt_analyzer.analis_optimization_flow(opt_config,False)
analyzation_report_arr

2020-02-12 00:00:00 strategy [INFO] SELL_SHORT at 1.0921800136566162
2020-03-02 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.1052532196044922
2020-03-03 00:00:00 strategy [INFO] BUY at 1.1144297122955322
2020-03-19 00:00:00 strategy [INFO] Close SELL order at 1.0943433046340942
2020-03-20 00:00:00 strategy [INFO] SELL_SHORT at 1.0658481121063232
2020-05-22 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.095242261886597
2020-05-29 00:00:00 strategy [INFO] BUY at 1.1078122854232788
2020-09-25 00:00:00 strategy [INFO] Close SELL order at 1.1673650741577148
2020-09-28 00:00:00 strategy [INFO] SELL_SHORT at 1.1635076999664309
2020-11-10 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.1823265552520752
2020-12-03 00:00:00 strategy [INFO] BUY at 1.2113430500030518
2024-03-02 14:55:33,243 absTradingSimulatior[{'name': 'Example Strategy', 'v': 'v1'}] [INFO] Convert log into report
2024-03-02 14:55:33,243 absTradingSimulatior[{'name': 'Example Strategy', 'v': 'v1'}] [INFO]

[OptimizationAnalyzReport(optimization={'abs_cap_log': {datetime.datetime(2020, 1, 1, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 2, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 3, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 6, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 7, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 8, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 9, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 10, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 13, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 14, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 15, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 16, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 17, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 20, 0, 0): 1000000,
                  datetime.datetime(2020, 1, 21, 0, 0): 100

# Report view

In [11]:
from src.optimization_analyzer.model import OptimizationAnalyzReport

In [12]:
OptimizationAnalyzReport.flow_to_dataframe(analyzation_report_arr)

capital  \
                                                                                                               max_fall   
                                                                                                                forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)             
2020-01-01                   2020-12-26                     2020-12-26              2021-03-26                -0.002940   
2020-03-31                   2021-03-26                     2021-03-26              2021-06-24                -0.004024   
2020-06-29                   2021-06-24                     2021-06-24              2021-09-22                -0.003879   
2020-09-27                   2021-09-22                     2021-09-22              2021-12-21                -0.001986   
2020-12-26                   2021-12-21                     2021-12-21              2022-03-21                -0.005643   
2021-03-26                   2022-03-21                     2022-03-21              2022-06-19                -0.004131   
2021-06-24                   2022-06-19                     2022-06-19              2022-09-17                -0.003671   
2021-09-22                   2022-09-17                     2022-09-17              2022-12-16                -0.005428   
2021-12-21                   2022-12-16                     2022-12-16              2023-03-16                -0.003091   
2022-03-21                   2023-03-16                     2023-03-16              2023-06-14                -0.002159   
2022-06-19                   2023-06-14                     2023-06-14              2023-09-12                -0.002588   
2022-09-17                   2023-09-12                     2023-09-12              2023-12-11                -0.002778   

                                                                                                                            \
                                                                                                                             
                                                                                                              optimization   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                
2020-01-01                   2020-12-26                     2020-12-26              2021-03-26                   -0.010879   
2020-03-31                   2021-03-26                     2021-03-26              2021-06-24                   -0.005269   
2020-06-29                   2021-06-24                     2021-06-24              2021-09-22                   -0.005968   
2020-09-27                   2021-09-22                     2021-09-22              2021-12-21                   -0.003927   
2020-12-26                   2021-12-21                     2021-12-21              2022-03-21                   -0.004074   
2021-03-26                   2022-03-21                     2022-03-21              2022-06-19                   -0.004536   
2021-06-24                   2022-06-19                     2022-06-19              2022-09-17                   -0.004550   
2021-09-22                   2022-09-17                     2022-09-17              2022-12-16                   -0.005245   
2021-12-21                   2022-12-16                     2022-12-16              2023-03-16                   -0.010180   
2022-03-21                   2023-03-16                     2023-03-16              2023-06-14                   -0.010139   
2022-06-19                   2023-06-14                     2023-06-14              2023-09-12                   -0.008044   
2022-09-17                   2023-09-12                     2023-09-12              2023-12-11                   -0.004325   

                                                                                                                         \
   